Referred doc- https://huggingface.co/learn/nlp-course/chapter6/8?fw=pt#building-a-bpe-tokenizer-from-scratch

In [ ]:
pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.7 MB/s eta 0:00:00


In [ ]:
import huggingface_hub

list(huggingface_hub.list_datasets())

[DatasetInfo(id='acronym_identification', author=None, sha='15ef643450d589d5883e289ffadeb03563e80a9e', created_at=datetime.datetime(2022, 3, 2, 23, 29, 22, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 1, 9, 11, 39, 57, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=1004, likes=17, paperswithcode_id='acronym-identification', tags=['task_categories:token-classification', 'annotations_creators:expert-generated', 'language_creators:found', 'multilinguality:monolingual', 'size_categories:10K<n<100K', 'source_datasets:original', 'language:en', 'license:mit', 'acronym-identification', 'arxiv:2010.14678', 'region:us'], card_data=None, siblings=None),
 DatasetInfo(id='ade_corpus_v2', author=None, sha='4ba01c71687dd7c996597042449448ea312126cf', created_at=datetime.datetime(2022, 3, 2, 23, 29, 22, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 1, 9, 11, 42, 58, tzinfo=datetime.timezone.utc), private=False, gated=F

In [ ]:
len(list(huggingface_hub.list_datasets()))

101377

In [ ]:
from datasets import load_dataset

# Load AG News dataset
ag_news_dataset = load_dataset("ag_news", split = "train[:15000]")

ag_news_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 15000
})

In [ ]:
ag_news_dataset['text'][1]

'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.'

In [ ]:
text_samples = [example['text'] for example in ag_news_dataset]

text_samples[:5]

["Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.',
 "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.",
 'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.',
 'Oil prices soar to all-time record, posing new menace to US economy (A

In [ ]:
len(text_samples)

15000

Let’s now build a GPT-2 tokenizer. We start by initializing a Tokenizer with a BPE model

More precisely, the library is built around a central Tokenizer class with the building blocks regrouped in submodules:

- normalizers contains all the possible types of Normalizer you can use (complete list here).
- pre_tokenizers contains all the possible types of PreTokenizer you can use (complete list here).
- models contains the various types of Model you can use, like BPE, WordPiece, and Unigram (complete list here).
- trainers contains all the different types of Trainer you can use to train your model on a corpus (one per type of model; complete list here).
- post_processors contains the various types of PostProcessor you can use (complete list here).
- decoders contains the various types of Decoder you can use to decode the outputs of tokenization (complete list here).

In [ ]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.BPE())

Also like for BERT, we could initialize this model with a vocabulary if we had one (we would need to pass the vocab and merges in this case), but since we will train from scratch, we don’t need to do that. We also don’t need to specify an unk_token because GPT-2 uses byte-level BPE, which doesn’t require it.

GPT-2 does not use a normalizer, so we skip that step and go directly to the pre-tokenization. The option we added to ByteLevel here is to not add a space at the beginning of a sentence (which is the default otherwise).

In [ ]:
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space = False)

tokenizer.pre_tokenizer.pre_tokenize_str("We're checking pre-tokenization step.")

[('We', (0, 2)),
 ("'re", (2, 5)),
 ('Ġchecking', (5, 14)),
 ('Ġpre', (14, 18)),
 ('-', (18, 19)),
 ('tokenization', (19, 31)),
 ('Ġstep', (31, 36)),
 ('.', (36, 37))]

In [ ]:
tokenizer.pre_tokenizer.pre_tokenize_str("The pièce de résistance was the chef's Special Dessert.")

[('The', (0, 3)),
 ('ĠpiÃ¨ce', (3, 9)),
 ('Ġde', (9, 12)),
 ('ĠrÃ©sistance', (12, 23)),
 ('Ġwas', (23, 27)),
 ('Ġthe', (27, 31)),
 ('Ġchef', (31, 36)),
 ("'s", (36, 38)),
 ('ĠSpecial', (38, 46)),
 ('ĠDessert', (46, 54)),
 ('.', (54, 55))]

The option we added to ByteLevel here is to not add a space at the beginning of a sentence (which is the default otherwise). We can have a look at the pre-tokenization of an example text like before


Next is the model, which needs training. For GPT-2, the only special token is the end-of-text token

In [ ]:
trainer = trainers.BpeTrainer(vocab_size = 20000, special_tokens = ["<|endoftext|>"])

tokenizer.train_from_iterator(text_samples, trainer = trainer)

Like with the WordPieceTrainer, as well as the vocab_size and special_tokens, we can specify the min_frequency if we want to, or if we have an end-of-word suffix (like </w>), we can set it with end_of_word_suffix.

Let’s have a look at the tokenization of a sample text

In [ ]:
encoding = tokenizer.encode("We're checking pre-tokenization step.")

encoding

Encoding(num_tokens=11, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
print("Encoding_ids :", encoding.ids)

print("Encoding_tokens :", encoding.tokens)

print("Encoding_offsets :", encoding.offsets)

Encoding_ids : [5024, 3662, 3887, 103, 760, 11, 375, 14194, 6943, 2079, 12]
Encoding_tokens : ['We', "'re", 'Ġcheck', 'ing', 'Ġpre', '-', 'to', 'ken', 'ization', 'Ġstep', '.']
Encoding_offsets : [(0, 2), (2, 5), (5, 11), (11, 14), (14, 18), (18, 19), (19, 21), (21, 24), (24, 31), (31, 36), (36, 37)]


### TODO Recording:

- First run with the default value of ByteLevel()
- Then come back and run with ByteLevel(trim_offsets = False)
- Show the difference

In [ ]:
# tokenizer.post_processor = processors.ByteLevel()

tokenizer.post_processor = processors.ByteLevel(trim_offsets = False)

The trim_offsets = False option indicates to the post-processor that we should leave the offsets of tokens that begin with ‘Ġ’ as they are: this way the start of the offsets will point to the space before the word, not the first character of the word (since the space is technically part of the token). Let’s have a look at the result with the text we just encoded, where 'Ġchecking' and 'Ġstep' are the tokens at index 2 and 8 respectively:

In [ ]:
sentence = "We're checking pre-tokenization step."

encoding = tokenizer.encode(sentence)

print("Encoding_offsets :", encoding.offsets)

Encoding_offsets : [(0, 2), (2, 5), (5, 11), (11, 14), (14, 18), (18, 19), (19, 21), (21, 24), (24, 31), (31, 36), (36, 37)]


In [ ]:
start, end = encoding.offsets[2]

sentence[start:end]

' check'

In [ ]:
start, end = encoding.offsets[9]

sentence[start:end]

' step'

Finally, we add a byte-level decoder:

In [ ]:
tokenizer.decoder = decoders.ByteLevel()

tokenizer.decode(encoding.ids)

"We're checking pre-tokenization step."

and we can double-check it works properly:

 We can save the tokenizer like before, and wrap it in a PreTrainedTokenizerFast or GPT2TokenizerFast if we want to use it in 🤗 Transformers

In [ ]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object = tokenizer,
    bos_token = "<|endoftext|>",
    eos_token = "<|endoftext|>",
)

wrapped_tokenizer.tokenize("We're checking pre-tokenization step.")

['We',
 "'re",
 'Ġcheck',
 'ing',
 'Ġpre',
 '-',
 'to',
 'ken',
 'ization',
 'Ġstep',
 '.']

Alternatively

In [ ]:
from transformers import GPT2TokenizerFast

wrapped_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

wrapped_tokenizer.tokenize("We're checking pre-tokenization step.")

['We',
 "'re",
 'Ġcheck',
 'ing',
 'Ġpre',
 '-',
 'to',
 'ken',
 'ization',
 'Ġstep',
 '.']